# Dust emission

`Synthesizer` has the capability of creating a dust emission spectrum using a simple energy balance approach. This can be done using dust emission models which can be handed to an ``EmissionModel`` to produce a dust emission spectrum and combine it with any other spectra in the model.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from synthesizer.emission_models.dust.emission import (
    Blackbody,
    Casey12,
    Greybody,
    IR_templates,
)
from unyt import Angstrom, K, Lsun, Msun, um

Before demonstrating any of these models we first need a wavelength array to demonstrate them with.

In [ ]:
lam = 10 ** (np.arange(3.0, 8.0, 0.01)) * Angstrom

## Blackbody

First let's explore simple blackbody models. By default a dust emission model provides a normalised spectrum.

In [ ]:
for T in [10, 25, 50, 100, 1000]:
    model = Blackbody(T * K)
    sed = model.get_spectra(lam)
    plt.plot(np.log10(sed.lam), sed.luminosity, label=f"{T} K")

plt.legend()
plt.show()

## Greybody

A greybody model is a simple extension of the blackbody model where the emissivity is not unity.

In [ ]:
for T in [10, 25, 50, 100]:
    model = Greybody(T * K, 1.6)
    sed = model.get_spectra(lam)
    plt.plot(np.log10(sed.lam), sed.luminosity, label=f"{T} K")

plt.legend()
plt.show()

## Casey12

The Casey12 model contains the model detailed in [Casey et al. (2012)](https://ui.adsabs.harvard.edu/abs/2012MNRAS.425.3094C/abstract).

In [ ]:
for T in [10, 25, 50, 100]:
    model = Casey12(T * K, 1.6, 2.0)
    sed = model.get_spectra(lam)
    plt.plot(np.log10(sed.lam), sed.luminosity, label=f"{T} K")

plt.legend()
plt.show()

## Adding CMB heating
For high redshift observations the effect of CMB heating is important. To add CMB heating we simply flag it on. Below we compare the dust emission spectrum with and without CMB heating.

In [ ]:
for T in [20, 25, 50, 100]:
    model = Casey12(T * K, 1.6, 2.0)
    model_cmb = Casey12(T * K, 1.6, 2.0, cmb_heating=True, z=7)
    sed = model.get_spectra(lam)
    sed_cmb = model_cmb.get_spectra(lam)
    L_ir_ratio = sed_cmb.measure_window_luminosity(
        window=[8, 1000] * um
    ) / sed.measure_window_luminosity(
        window=[8, 1000] * um
    )  # same as model_cmb.cmb_factor
    plt.scatter(
        T, L_ir_ratio, label=f"{np.around(model_cmb.temperature_z, 2)}"
    )

plt.axhline(y=1)
plt.grid(ls="dotted")
plt.xlabel("Input dust temperature")
plt.ylabel(r"L$_{\rm IR}$ ratio")
plt.legend()
plt.show()

### IR Templates - Draine & Li 2007 dust models

We also implemented the more complex model present in [Draine & Li 2007](https://ui.adsabs.harvard.edu/abs/2007ApJ...657..810D/abstract). This model is more complex and requires a grid to be loaded in.

Begin by reading in the DL07 grids, which have been created by downloading the ASCII DL07 files and running 
```
from synthesizer.utils import process_dl07_to_hdf5
process_dl07_to_hdf5()
```

However, you can download these preprocessed grids using the ``synthesizer-download`` command line tool.

```bash
synthesizer-download -d /path/to/destination --dust-grid
```

In [ ]:
from synthesizer.grid import Grid

grid_name = "MW3.1"
grid_dir = "../../../tests/test_grid/"
grid = Grid(grid_name, grid_dir=grid_dir, read_spectra=True, read_lines=False)

In [ ]:
print(grid.axes)

Below we show how to use the DL07 model directly and plot the dust emission spectrum for a number of dust luminosities. (Note however that you can also pass these models to an ``EmissionModel`` and they will be used automatically.)

In [ ]:
for mdust in [1e7, 1e8, 5e8, 1e9, 5e9]:
    model = IR_templates(
        grid, mdust=mdust * Msun, ldust=1e11 * Lsun, verbose=False
    )
    sed = model.get_spectra(lam)
    plt.plot(
        np.log10(sed.lam),
        np.log10(sed.luminosity),
        label="{:.1e} Msun, <U>={}".format(mdust, model.u_avg),
    )

plt.xlabel("log10(lam/AA)")
plt.ylabel("log10(lnu/(erg/s))")
plt.ylim(40, 45)
plt.legend()
plt.show()